In [1]:
%matplotlib inline
import matplotlib as mlp
mlp.rcParams['figure.dpi'] = 100

# diasbling warning messages
import warnings
warnings.filterwarnings("ignore")

# importing key libraries
import pandas as pd
pd.set_option('display.max_rows', None)
import numpy as np

# importing wrangle/acquire module
import wrangle
from wrangle import get_zillow_dataset

# importing data visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns 
sns.set()

# importing mysql
import env 
from env import user, password, host, get_connection

----
### **``Clustering Module: Data Wrangling Exercises``**



<u>**Acquire and Summarize**</u>

1. Acquire data from the cloud database

You will want to end with a single dataframe. Include the logerror field and all other fields related to the properties that are available. You will end up using all the tables in the database.

**Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.** 

Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction. 
  
(Hint: read the docs for the .duplicated method) - Only include properties that have a latitude and longitude value.

2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. 
   - Run the function and document takeaways from this on how you want to handle missing values.


----

In [2]:
# pulling zillow dataset and creating a function for future pull:
# query = '''
# SELECT *
# FROM properties_2017
#         RIGHT JOIN (SELECT parcelid, any_value(logerror), MAX(transactiondate) AS maxtransaction_date
#             FROM predictions_2017
#                 GROUP BY (predictions_2017.parcelid)) AS table2 USING (parcelid)
#                     WHERE maxtransaction_date < 2018 '''

# url = get_connection(user, password, host, "zillow")

In [3]:
# df = pd.read_sql(query, url)
# df.head()

In [4]:
# getting cached zillow dataset
df = get_zillow_dataset()
df.head()

,parcelid,any_value(logerror),maxtransaction_date,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,14297519,0.025595,2017-01-01,1727539,NaN,NaN,NaN,3.5,4.0,NaN,...,NaN,NaN,485713.0,1023282.0,2016.0,537569.0,11013.72,NaN,NaN,6.059063e+13
1,17052889,0.055619,2017-01-01,1387261,NaN,NaN,NaN,1.0,2.0,NaN,...,1.0,NaN,88000.0,464000.0,2016.0,376000.0,5672.48,NaN,NaN,6.111001e+13
2,14186244,0.005383,2017-01-01,11677,NaN,NaN,NaN,2.0,3.0,NaN,...,1.0,NaN,85289.0,564778.0,2016.0,479489.0,6488.30,NaN,NaN,6.059022e+13
3,12177905,-0.103410,2017-01-01,2288172,NaN,NaN,NaN,3.0,4.0,NaN,...,NaN,NaN,108918.0,145143.0,2016.0,36225.0,1777.51,NaN,NaN,6.037300e+13
4,10887214,0.006940,2017-01-01,1970746,1.0,NaN,NaN,3.0,3.0,NaN,...,NaN,NaN,73681.0,119407.0,2016.0,45726.0,1533.89,NaN,NaN,6.037124e+13


In [5]:
# checking the shape
df.shape

(77413, 61)

In [6]:
# renaming any_value(logerror) and maxtransaction_date columns
df = df.rename(columns = {"any_value(logerror)": "zestimate_error", "maxtransaction_date": "transaction_date"})
df.head()

,parcelid,zestimate_error,transaction_date,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,14297519,0.025595,2017-01-01,1727539,NaN,NaN,NaN,3.5,4.0,NaN,...,NaN,NaN,485713.0,1023282.0,2016.0,537569.0,11013.72,NaN,NaN,6.059063e+13
1,17052889,0.055619,2017-01-01,1387261,NaN,NaN,NaN,1.0,2.0,NaN,...,1.0,NaN,88000.0,464000.0,2016.0,376000.0,5672.48,NaN,NaN,6.111001e+13
2,14186244,0.005383,2017-01-01,11677,NaN,NaN,NaN,2.0,3.0,NaN,...,1.0,NaN,85289.0,564778.0,2016.0,479489.0,6488.30,NaN,NaN,6.059022e+13
3,12177905,-0.103410,2017-01-01,2288172,NaN,NaN,NaN,3.0,4.0,NaN,...,NaN,NaN,108918.0,145143.0,2016.0,36225.0,1777.51,NaN,NaN,6.037300e+13
4,10887214,0.006940,2017-01-01,1970746,1.0,NaN,NaN,3.0,3.0,NaN,...,NaN,NaN,73681.0,119407.0,2016.0,45726.0,1533.89,NaN,NaN,6.037124e+13


In [7]:
# checking the dataframe info
df.dtypes.sort_values()

parcelid                          int64
id                                int64
poolsizesum                     float64
pooltypeid10                    float64
pooltypeid2                     float64
pooltypeid7                     float64
propertylandusetypeid           float64
rawcensustractandblock          float64
regionidcity                    float64
regionidcounty                  float64
regionidneighborhood            float64
regionidzip                     float64
roomcnt                         float64
storytypeid                     float64
threequarterbathnbr             float64
typeconstructiontypeid          float64
unitcnt                         float64
yardbuildingsqft17              float64
yardbuildingsqft26              float64
yearbuilt                       float64
numberofstories                 float64
fireplaceflag                   float64
structuretaxvaluedollarcnt      float64
taxvaluedollarcnt               float64
assessmentyear                  float64


In [8]:
# only include properties that have a latitude and longitude value
# let's check how many records have both latitude and longitude features missing

df[["latitude", "longitude"]].isnull().sum()

latitude     33
longitude    33
dtype: int64

In [9]:
# df observasation where these two are missing

df[(df["latitude"].isnull() == True) & (df["longitude"].isnull() == True)]

,parcelid,zestimate_error,transaction_date,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
387,10813418,-0.008083,2017-01-03,2985044,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1080,12133448,-1.062436,2017-01-05,2982537,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3561,11905748,0.084014,2017-01-18,2984012,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3711,12961343,0.559867,2017-01-18,2983937,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3767,12006414,-0.795601,2017-01-18,2984982,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4888,12039176,0.175751,2017-01-24,2984396,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8837,11905738,-0.050252,2017-02-09,2983854,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9272,12185469,-0.447673,2017-02-10,2982853,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10409,11905737,-0.029747,2017-02-16,2983837,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11363,10855501,0.018408,2017-02-22,2983339,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
